In [1]:
# For Kmeans algorithm. See PGM otherwise for handcrafted class
from sklearn.cluster import KMeans
# For loading matlab matrix file
from scipy.io import loadmat

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
%precision %.5f
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Palatino']})

In [ ]:
def SC(n, W):
    """
    Spectral Clustering algorithm.
    :param n: number of clusters
    :param W: affinity matrix NxN (N : number of points)
    :return : Segmentation of the data in n groups
    """
    # 
    # 
    # 
    # 
    # 